In [1]:
import sys
if "../src/" not in sys.path:
    sys.path.append("../src/")
from roplot import FieldPowerPlay, Robot, StateRecord, DeclarativeBase
from datetime import datetime
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, desc
from sqlalchemy.orm import Session

from shapely import Point

In [2]:
# SQLITE Handles concurrency well so long as there's not concurrent writes. 
# So this should work fine while monitoring since we're only reading.
fpp = FieldPowerPlay()

DATA_PATH='../src/data'
TODAY = datetime.now().strftime("%Y-%m-%d")
ENGINE = create_engine(f"sqlite:///{DATA_PATH}/{TODAY}.sqlite")

session = Session(ENGINE)
def update_position(r):
    sr = session.query(StateRecord).filter_by(type="CombinedLocalizer").order_by(desc( StateRecord.id )).limit(1).first()
#     sr.record
    r.position = (sr.record['x'], sr.record['y'])
    r.heading = sr.record['heading']
    print(sr.record['secondsSinceVuforia'])

# Robot Monitor and Draw cell works
If the state is being updated in a sqlite database using a `monitor.py`, the following code block draws the robot in the right place.  Next up: Figure out how to make it update easily.

In [139]:
from branca.element import Figure
import geopandas as gpd
import folium
crs = "Simple"
m2 = folium.Map(location=[72, 72], zoom_start=2, min_zoom=1, max_zoom=4, tiles=None, crs=crs)
fpp.add_to(m2, show_junctions=True)

# folium.GeoJson(gpd.GeoSeries(field).to_json()).add_to(m2)
# 17, 16 is the SIZE of the robot. 
r = Robot((17, 16))

update_position(r)
print(r.position)
m2.add_child(r, name="Robot")
m2

0.078790841
POINT (36.98787166347143 37.447537288235154)


In [90]:
sr = session.query(StateRecord).filter_by(type="CombinedLocalizer").order_by(desc( StateRecord.id )).limit(1).first()
print(sr.type, "heading: {heading:02}+/-{headingUncertainty:02}".format(**sr.record))

CombinedLocalizer heading: 92.4191751998327+/-0.2128992808581437
